# File formats in forecats
---
En este notebook se explican los diferentes formato de archivos usados por los modelos de predicción climática.

> **GRIB:**
>
> Es un formato utilizado para almacenar datos meteorológicos definido por la Organización Meteorológica Mundial para almacenar y distribuir datos reticulados. Sus siglas significan: "General Regularly distributed Information in Binary form" por tanto es un archivo en formato binario para incrementar la eficiencia a la hora de ser almacenados. Estos archivos están conformados por mensajes GRIB que se concatenan para formar un archivo GRIB. Los archivos GRIB suelen tener la extensión .grib, .grb o .gb.
>
> Puede encontrar más información en la página de [ECMWF](https://confluence.ecmwf.int/display/CKB/What+are+GRIB+files+and+how+can+I+read+them#heading-Introduction) 

> **PT:**
>
> Es un formato que se asocia principalmente con `PyTorch`. Los archivos .pt en PyTorch generalmente contienen modelos entrenados, parámetros de red neuronal, pesos y otros datos relacionados con el modelo. Este formato es ampliamente utilizado para guardar y cargar modelos entrenados en PyTorch.

> **NC:**
>
> Este formato lleva el nombre de Network Common Data Form (NetCDF) es un conjunto de bibliotecas de software y formatos de datos que permiten crear, acceder y compartir datos científicos orientados a matrices. 

## GRIB to NC
---
Para cargar un .GRIB con xarray es necesario instalar `cfgrib`

In [ ]:
# !pip install cfgrib

In [17]:
import xarray as xr

In [18]:
current_dir = "C:\\Users\\gcuervo\\OneDrive - Universidad de Las Palmas de Gran Canaria\\Documents\\Doctorado\\DB\\Data_formats\\"
path_grib_file = current_dir + "download.grib"



data = xr.open_dataset(path_grib_file, engine='cfgrib')

Ignoring index file 'C:\\Users\\gcuervo\\OneDrive - Universidad de Las Palmas de Gran Canaria\\Documents\\Doctorado\\DB\\Data_formats\\download.grib.923a8.idx' incompatible with GRIB file


In [19]:
data

<xarray.Dataset>
Dimensions:        (latitude: 721, longitude: 1440)
Coordinates:
    number         int32 ...
    time           datetime64[ns] ...
    step           timedelta64[ns] ...
    isobaricInhPa  float64 ...
  * latitude       (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude      (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time     datetime64[ns] ...
Data variables:
    t              (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-02-12T21:13 GRIB to CDM+CF via cfgrib-0.9.1...

Para convertir un GRIB a NC usamos el método ``.to_netcdf()``

In [20]:
data.to_netcdf(current_dir + 'netcdf_file.nc')

## NC to Pytorch
---
Desafortunadamente Pytorch no tiene métodos nativos para cargar datos tipo NC por lo que se utilizan otros paquetes.

En estos foros discuten sobre como alimentar modelos de ML con datos NetCDF 
[NC to tensorflow](https://www.noahbrenowitz.com/post/loading_netcdfs/)
[Discusión](https://discourse.pangeo.io/t/favorite-way-to-go-from-netcdf-xarray-to-torch-tf-jax-et-al/2663)

El paquete ``Xbatcher`` pare ser la herramiente más eficiente para dividir un NC en batches para alimentar dataloaders de ``Pytorch``
[Xbatcher](https://figshare.com/articles/presentation/Xbatcher_-_A_Python_Package_That_Simplifies_Feeding_Xarray_Data_Objects_to_Machine_Learning_Libraries/22264072/1)

In [ ]:
# !pip install xbatcher

In [11]:
import xbatcher

In [8]:
torch.tensor(r"C:\Users\gcuervo\OneDrive - Universidad de Las Palmas de Gran Canaria\Documents\Doctorado\PhD_repo\netcdf_file.nc")

TypeError: new(): invalid data type 'str'

In [1]:
import pygrib

In [4]:
with pygrib.open(r"C:\Users\gcuervo\OneDrive - Universidad de Las Palmas de Gran Canaria\Documents\Doctorado\PhD_repo\download.grib") as grib:
    # Create an empty dictionary to hold data:
    data = {}
    # Iterate over messages in grib file:
    for message in grib:
        # Extract key names:
        date = str(message.date)
        print(date)
        hour = '0' + \
            str(message.hour) if message.hour < 10 else str(message.hour)
        time = date+hour
        level = message.level
        variable = message.name
        # Add data and missing keys to nested dictionary:
        if time not in data:
            data[time] = {}
        if level not in data[time]:
            data[time][level] = {}
        data[time][level][variable] = message.values

20080101


In [5]:
data.keys()

dict_keys(['2008010112'])

: 